In [1]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import pytz

In [3]:
df = pd.read_excel('./data/Open_Seats_Report.xlsx', skiprows=1)
df.head()

,Learning Content,Catalog,Learning Content Number,Course Scheduled Time Zone,Viewer Time Zone Start Date,Viewer Time Zone End Date,Viewer Time Zone Registration & Prerequisite Completion Deadline,Availability Status,Available Capacity,Primary Instructors,Price in USD,Price in Training Credits,Language,Course Offering Locations
0,HCM Services Practical - Learn Virtual,Services,S-02403,GMT+10:00 Eastern Australia Time (Sydney),"Mar 11, 2024 09:00:00 am","Mar 15, 2024 05:00:00 pm","Mar 06, 2024 10:00:00 am",Open,Yes,Lalitha TS,1000,1.25,English (United States),NaN
1,Time Tracking Consulting Core - Learn Virtual,Services,S-01779,GMT+08:00 Singapore Standard Time (Singapore),"Mar 11, 2024 12:00:00 pm","Mar 15, 2024 09:00:00 pm","Mar 06, 2024 01:00:00 pm",Open,Yes,Jack Loke,2000,2.50,English (United States),NaN
2,Recruiting Fundamentals - Learn Virtual,Customer,S-02401,GMT+08:00 Singapore Standard Time (Singapore),"Mar 11, 2024 12:00:00 pm","Mar 14, 2024 08:00:00 pm","Mar 06, 2024 01:00:00 pm",Open,Yes,Denise Ng,3200,4.00,English (United States),NaN
3,Recruiting Consulting Core - Learn Virtual,Services,S-02402,GMT+08:00 Singapore Standard Time (Singapore),"Mar 11, 2024 12:00:00 pm","Mar 15, 2024 09:00:00 pm","Mar 06, 2024 01:00:00 pm",Open,Yes,Denise Ng,2000,2.50,English (United States),NaN
4,Calculated Fields - Learn Virtual,Customer,S-02399,GMT+08:00 China Standard Time (Shanghai),"Mar 11, 2024 12:00:00 pm","Mar 12, 2024 05:00:00 pm","Mar 06, 2024 01:00:00 pm",Open,Yes,Celia Chan （陳雅莉）,800,1.00,English (United States),NaN


In [4]:
# Create new date columns (don't overwrite existing ones)
df['Start Date'] = pd.to_datetime(df['Viewer Time Zone Start Date'], format='mixed')
df['End Date'] = pd.to_datetime(df['Viewer Time Zone End Date'], format='mixed')
df['Registration Deadline'] = pd.to_datetime(df['Viewer Time Zone Registration & Prerequisite Completion Deadline'], format='mixed')


In [5]:
# Extract city names
df['City'] = df['Course Scheduled Time Zone'].str.extract(pat=r'\((.*?)\)')

## Get city names (use this to create a mapping to relevant time zones)
print(np.sort(df.City.unique()))  # Show cities 
# print(pytz.all_timezones)  # Show timezones

# Use the printed information to create a timezone mapping
timezones = {
    'Berlin': 'Europe/Berlin',
    'Chicago': 'America/Chicago', 
    'Denver':'America/Denver', 
    'Kolkata': 'Asia/Kolkata', 
    'London': 'Europe/London', 
    'Los Angeles': 'America/Los_Angeles',
    'New York': 'America/New_York', 
    'Paris': 'Europe/Paris', 
    'Seoul': 'Asia/Seoul', 
    'Shanghai': 'Asia/Shanghai', 
    'Singapore': 'Asia/Singapore', 
    'Sydney': 'Australia/Sydney',
    'Tokyo': 'Asia/Tokyo'
}

# Create a new column with city mapping over to timezone
df['Timezone'] = df['City'].map(timezones)

['Berlin' 'Chicago' 'Denver' 'Kolkata' 'London' 'Los Angeles' 'New York'
 'Paris' 'Seoul' 'Shanghai' 'Singapore' 'Sydney' 'Tokyo']


In [6]:
# Take old timezone and time and convert to Sydney time
def convert_timezone(row, column):
    """Converts time in any location to a Sydney time"""
    old_timezone = pytz.timezone(row['Timezone'])  # changes depending on row
    new_timezone = pytz.timezone('Australia/Sydney')  # stays constant
    old_datetime = pd.to_datetime(row[column])  # changes depending on row and column we choose
    new_datetime = old_timezone.localize(old_datetime).astimezone(new_timezone)
    return new_datetime

In [7]:
# Dynamically create new Sydney-timezoned columns using my function
for column in ['Start Date', 'End Date', 'Registration Deadline']:
    df[f'Sydney {column}'] = df.apply(lambda row: convert_timezone(row, column), axis=1)

In [8]:
kept_columns = ['Learning Content', 
                'Availability Status', 
                'Available Capacity',
                'Sydney Start Date',	
                'Sydney End Date',
                'Sydney Registration Deadline']

new = df[kept_columns].copy()

In [9]:
# Now we have a new dataset with Sydney Adjusted times
new

,Learning Content,Availability Status,Available Capacity,Sydney Start Date,Sydney End Date,Sydney Registration Deadline
0,HCM Services Practical - Learn Virtual,Open,Yes,2024-03-11 09:00:00+11:00,2024-03-15 17:00:00+11:00,2024-03-06 10:00:00+11:00
1,Time Tracking Consulting Core - Learn Virtual,Open,Yes,2024-03-11 15:00:00+11:00,2024-03-16 00:00:00+11:00,2024-03-06 16:00:00+11:00
2,Recruiting Fundamentals - Learn Virtual,Open,Yes,2024-03-11 15:00:00+11:00,2024-03-14 23:00:00+11:00,2024-03-06 16:00:00+11:00
3,Recruiting Consulting Core - Learn Virtual,Open,Yes,2024-03-11 15:00:00+11:00,2024-03-16 00:00:00+11:00,2024-03-06 16:00:00+11:00
4,Calculated Fields - Learn Virtual,Open,Yes,2024-03-11 15:00:00+11:00,2024-03-12 20:00:00+11:00,2024-03-06 16:00:00+11:00
...,...,...,...,...,...,...
1048,Workday Extend Fundamentals - Learn Virtual,Open,Yes,2024-05-31 17:00:00+10:00,2024-06-01 22:00:00+10:00,2024-05-28 17:00:00+10:00
1049,Calculated Fields - Learn Virtual,Open,Yes,2024-05-31 19:00:00+10:00,2024-06-02 00:00:00+10:00,2024-05-28 19:00:00+10:00
1050,Workday Report Designer (BIRT) - Learn Virtual,Open,Yes,2024-05-31 19:00:00+10:00,2024-06-02 00:00:00+10:00,2024-05-28 19:00:00+10:00
1051,Workday Studio Consulting Core - Learn Virtual,Open,Yes,2024-06-05 01:00:00+10:00,2024-06-14 07:00:00+10:00,2024-05-31 01:00:00+10:00


In [10]:
new.to_csv('./data/open_seats_report_adjusted.csv', index=False)